# Shopee - The Best Coder Challenger 2019 - Round 2

## Setting up

Import necessary libraries.
- Just import __Numpy__ for fun
- __Pandas__ to work with the provided data
- __NetworkX__ to create graph

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

Import data as __Pandas dataframes__. Using `dtype='str'` so that Pandas won't read numbers as integers

In [2]:
df_order = pd.read_csv('orders.csv',dtype='str')
df_bank = pd.read_csv('bank_accounts.csv',dtype='str')
df_device = pd.read_csv('devices.csv',dtype='str')
df_card = pd.read_csv('credit_cards.csv',dtype='str')

See what the data looks like using `.head()`: showing the first few rows of the datafram

In [3]:
df_order.head(10)

,orderid,buyer_userid,seller_userid
0,1953278092,47388162,20822974
1,1953295120,26855196,16416890
2,1953306402,121296714,28477978
3,1953314712,131221930,72837912
4,1953381964,183398314,28423332
5,1953383372,69247284,62795594
6,1953451192,89510674,18186196
7,1953453596,198638326,7314602
8,1953479046,202508756,1988634
9,1953480060,130236220,77145726


In [4]:
df_device.head(10)

,userid,device
0,10499978,0Zu/mWZ4cFsvobYglgZEc5VSxwwajRwrm74pBuwnrTu=
1,54526742,zBMRKyz98iy5Z7sh+JIOdF612J1CB8ggvFBI6MP361q=
2,33152428,KXC0oY3MMrTG+bnlFFvBPXZC5PW8iw6kgWztENWdIJg=
3,55020986,anRXS7+zVgCGKz5C9X3dzewIhSdojGEi5TO07pb2y+K=
4,100471502,hbIARr+USwvjjzc8QUVtD5a/apR1DJX6P3fYvgDL4gW=
5,61471588,iVuRAqjJYPDrwq7W/OVP9EuiCLyPgOT9cv7ZmjDy4Gy=
6,62907954,Qy0OTEs+/dmv+aBN4Qw4mQScGsAAp435kHF69jdnQUm=
7,37491720,7IzAgA/z/+BpwyIwsGf4bfbUmZHpf+axxrM+ij8/FDg=
8,20756502,WdTSc6p/VKXhA6Hm+hwCk4xEmep7IiHC34sCIAf1oBW=
9,133160044,r7MzY5w6X7pz0kKqhAG4cIoU1CmcA/oG8BZ7CCBnK5E=


In [5]:
df_bank.head(10)

,userid,bank_account
0,21829134,923302000003892
1,95910542,011002023212822
2,96941876,0189303223
3,23452396,2280003199803
4,12647942,032002028484803
5,93351382,2200092211113
6,73673972,898802093909822
7,32609752,2390022092232
8,97851414,002902002222839
9,90467238,4032444322


In [6]:
df_card.head(10)

,userid,credit_card
0,2579938,832299xxxxxx4902|02-2019
1,2154902,322429xxxxxx3848|05-2020
2,5578604,322429xxxxxx1293|11-2016
3,2051728,388329xxxxxx4303|09-2016
4,6216212,322421xxxxxx4238|05-2018
5,7661324,388322xxxxxx8304|09-2018
6,9554574,829092xxxxxx2938|08-2019
7,11853094,388329xxxxxx3308|09-2018
8,7965348,322429xxxxxx8232|01-2021
9,7313766,390219xxxxxx0390|09-2016


## Prepare the data

Cleaning of data is not needed. The approach here is to create a __graph__ containing all available information. Each __node__ represents a data point, including:
- `userid`
- `device`
- `bank_account`
- `credit_card`

To know which node is connected to which, we look at the `df_device`, `df_bank`, and `df_card` dataframes. We connect the `userid` with his/her corresponding device, bank account of credit card according to the given data. In other words, each __row__ in the dataframe specifies a node connection, or an `edge` if we use the formal term.

In [7]:
G_device = nx.from_pandas_edgelist(df_device,source='userid',target='device')
G_bank = nx.from_pandas_edgelist(df_bank,source='userid',target='bank_account')
G_card = nx.from_pandas_edgelist(df_card,source='userid',target='credit_card')

Combine all separate graphs `G_device`, `G_bank`, and `G_card` into a single graph `G`

In [8]:
G = nx.compose_all([G_device,G_bank,G_card])

## Helper function

Define a helper function to help with checking _whether two nodes are connected_. The `nx.bidirectional_dijkstra()` will throw an error when no path is found, so I use the `try` syntax to handle the error:
- If there is an error, return 0 (meaning two nodes are __not connected__)
- If there is no error, return 1 (meaning two nodes are __connected__)

In [9]:
def is_connected(args):
    try:
        nx.bidirectional_dijkstra(G,args[0],args[1])
    except:
        return 0
    else:
        return 1

Testing the function, using a known-to-be-connected pair of `userid`s

In [10]:
nx.bidirectional_dijkstra(G,'70763052','70763052')

(0, ['70763052'])

In [11]:
is_connected(['35545436','70763052'])

1

## Execution

Use the `.apply()` method to apply the function to each row. The parameters:
- `axis=1`: to apply the function `is_connected` on each row instead of column (default behavior)
- `raw=True`: the whole row will be passed to the function as __1 object__, instead of passing as individual parameters

In [12]:
df_order['is_fraud'] = df_order.loc[:,['buyer_userid','seller_userid']].apply(is_connected,axis=1,raw=True)

Total number of matching found

In [13]:
df_order.is_fraud.sum()

478

Output the result to a `.csv` file

In [14]:
df_order.to_csv('output_with_graph.csv',index=False,columns=['orderid','is_fraud'])